In [38]:
import calendar
import time
import requests
import datetime
from bs4 import BeautifulSoup
from urllib import parse
import csv


In [59]:
# get html code, convert to plaintext
def getHTML(url):
    r = requests.get(url,verify=False)
    #r.status_code
    plain=r.text
    s = BeautifulSoup(plain, 'html.parser')
    return s

In [112]:

def getForum(s):
    # TODO: posts, topis, last post, subheading
    fields = ['fid', 'f_url','f_heading','f_subheading','num_topics','num_posts','last_post_date','last_post_uid']
    f_post=s.find_all('tr',{'class':'post'})
    rows = []
    for tags in f_post:
        link = tags.find_all('span',{'class':'forumheading'})[0]
        a = link.find_all('a')[0]
        fname = a.contents[0]
        url = a['href']
        fid = parse.parse_qs(parse.urlsplit(url).query)['f'][0]
        
        subheading = tags.find_all('span',{'class':'subforumheading'})[0].contents[0]
        topics = tags.find_all('td')[2].contents[0]
        posts = tags.find_all('td')[3].contents[0]
        last_date = tags.find_all('td')[4].contents[0]
        if last_date != '-':
            uid = parse.parse_qs(parse.urlsplit(tags.find_all('td')[4].contents[3]['data-remote']).query)['uid'][0]
        else:
            uid = '-'

        row = [fid, url, fname,subheading,topics,posts,last_date,uid]
        rows.append(row)

    return fields, rows

def writeToCSV(field, rows):
    with open('forum.csv', 'w') as f:
        csvwriter = csv.writer(f)
        csvwriter.writerow(fields)
        csvwriter.writerows(rows)
    f.close()


In [91]:
topsoup = getHTML('https://www.alzconnected.org/discussion.aspx')
topsoup.find_all('tr',{'class':'post'})[0].find_all('td')[4].contents

['Tuesday, February 16, 2021 9:25 PM',
 <br/>,
 'by ',
 <a data-refresh="true" data-remote="viewprofile.aspx?uid=2147590807" data-toggle="modal" href="#profile-modal" target="_self">dayn2nite2</a>,
 '\xa0',
 <a href="/discussion.aspx?g=posts&amp;t=2147555710&amp;page=-1" title="Go to last post"><img alt="" border="0" src="/WorkArea//threadeddisc/themes/icon_latest_reply.gif"/></a>]

In [67]:
forumsoup = getHTML('https://www.alzconnected.org/discussion.aspx?g=topics&f=151')
forumsoup.find_all('tr',{'class':'post'})[0]

<tr class="post">
<td><img alt="" src="/css/topic_announce.png" title="Announcement"/></td>
<td><span class="post_priority"></span><a class="post_link" href="/discussion.aspx?g=posts&amp;t=2147553329">ALZConnected Moderator’s Guidelines for Participation</a></td>
<td><a data-refresh="true" data-remote="viewprofile.aspx?uid=10052" data-toggle="modal" href="#profile-modal" target="_self">ALZConnected Moderator</a></td>
<td align="center">0</td>
<td align="center">6245</td>
<td align="center" class="smallfont">Friday, July 24, 2020 2:41 PM <a href="/discussion.aspx?g=posts&amp;page=-1&amp;t=2147553329#ekbottomfooter" title="Go to last post"><img alt="" border="0" src="/css/icon_latest_reply.gif"/></a></td>
</tr>

In [113]:
fields, rows = getForum(s)
writeToCSV(fields, rows)

In [115]:
for row in rows:
    forumURL = "https://www.alzconnected.org" + row[1]
    s = getHTML(forumURL)
    print(s)

<tr class="post">
<td><img alt="" src="/css/message.png" title="Topic"/></td>
<td><span class="post_priority"></span><a class="post_link" href="/discussion.aspx?g=posts&amp;t=2147551726">ALZHEIMER'S ASSOCIATION: AARP Coronavirus Tele-Town Hall is Today</a></td>
<td><a data-refresh="true" data-remote="viewprofile.aspx?uid=10052" data-toggle="modal" href="#profile-modal" target="_self">ALZConnected Moderator</a></td>
<td align="center">0</td>
<td align="center">5147</td>
<td align="center" class="smallfont">Thursday, March 26, 2020 11:43 AM <a href="/discussion.aspx?g=posts&amp;page=-1&amp;t=2147551726#ekbottomfooter" title="Go to last post"><img alt="" border="0" src="/css/icon_latest_reply.gif"/></a></td>
</tr><tr class="post">
<td><img alt="" src="/css/message.png" title="Topic"/></td>
<td><span class="post_priority"></span><a class="post_link" href="/discussion.aspx?g=posts&amp;t=2147551698">Alzheimer's Association: Covid-19 &amp; Tips for when you are providing care at home</a></td>